In [109]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score

In [110]:

# 1. Generate synthetic regression data
X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
df = pd.DataFrame(X,y)
#df.shape 
#X.shape
#y.shape
df.head()


,0,1,2,3,4,5,6,7,8,9
-62.955657,1.575876,0.891863,-0.726774,-0.047799,-0.657035,-1.113295,-0.877617,0.994558,-0.299696,-0.761760
108.721847,0.535459,1.266661,-1.779875,1.209010,-0.113158,-0.971062,1.064171,-0.555273,0.741320,-0.987523
-115.541354,-0.686279,0.157128,0.145836,-1.432671,-0.666849,0.176190,0.585299,0.197917,-0.404362,-0.607472
36.413165,0.925282,0.395914,0.676357,0.168651,1.574147,-0.117172,-0.037687,-0.345907,0.895796,-0.837381
130.839469,-0.558922,1.073632,1.565524,0.377212,0.132970,-1.026515,-0.065750,-0.700121,-1.523187,1.195047


In [111]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
gbr = GradientBoostingRegressor(
    n_estimators=100,    # Number of boosting stages
    learning_rate=0.1,   # Shrinks the contribution of each tree
    max_depth=3,        # Maximum depth of individual regression trees
    random_state=42
)

In [113]:

# 4. Fit the model to training data
gbr.fit(X_train, y_train)


,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [114]:
# 5. Make predictions and evaluate
predictions = gbr.predict(X_test)
#mse = mean_squared_error(y_test, predictions)
#print(f"Mean Squared Error: {mse:.4f}")
r2_score(y_test,predictions)


0.9268126639292507

# problem statement 
'''create a predictive model using gradient boosting to forecast housing prices based on various fetures such as square footage, number of bedroom,number odf bathroom and location.''''

In [53]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# step no 2

In [54]:
data={'Squarefootage':[1500,2000,1200,1800,1350],
      'Badroom':[3,4,2,3,3],
      'Bathroom':[2,2.5,1.5,2,2],
      'Location':['Suburb','City','Rural','City','Suburb'],
      'Price':[250000,300000,180000,280000,220000]}

In [55]:
df = pd.DataFrame(data)
print(df)

   Squarefootage  Badroom  Bathroom Location   Price
0           1500        3       2.0   Suburb  250000
1           2000        4       2.5     City  300000
2           1200        2       1.5    Rural  180000
3           1800        3       2.0     City  280000
4           1350        3       2.0   Suburb  220000


In [56]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# 3D scatter plot
fig = px.scatter_3d(
    df,
    x='Squarefootage',
    y='Badroom',
    z='Price',
    color='Location'  
)

fig.show()

In [57]:
# converting the location column to dummy 

df = pd.get_dummies(df,columns=['Location'],drop_first=True, dtype=int)
print(df)


   Squarefootage  Badroom  Bathroom   Price  Location_Rural  Location_Suburb
0           1500        3       2.0  250000               0                1
1           2000        4       2.5  300000               0                0
2           1200        2       1.5  180000               1                0
3           1800        3       2.0  280000               0                0
4           1350        3       2.0  220000               0                1


In [58]:
X= df.drop('Price',axis=1)
y= df['Price']


In [59]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [60]:
model = GradientBoostingRegressor( n_estimators=20,    # Number of boosting stages
    learning_rate=0.5,   # Shrinks the contribution of each tree
    max_depth=3,        # Maximum depth of individual regression trees
    random_state=42)

In [61]:
model.fit(X_train,y_train)


,loss,'squared_error'
,learning_rate,0.5
,n_estimators,20
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [62]:
y_pred=model.predict(X_test)

In [63]:
mse = mean_squared_error (y_test,y_pred)
print(mse)
 #ab shi predicction kar rha hai kal dekhenge ab ------

400001811.98325324


# ---------------- 3D SCATTER (ACTUAL) ----------------
fig = px.scatter_3d(
    df,
    x='Squarefootage',
    y='Badroom',
    z='Price',
    title='3D Scatter: Actual vs Predicted Price'
)

# ---------------- ADD PREDICTED POINTS (y_pred) ----------------
fig.add_scatter3d(
    x=X_test['Squarefootage'],
    y=X_test['Badroom'],
    z=y_pred,
    mode='markers',
    marker=dict(size=6),
    name='Predicted Price'
)

fig.show()

# small data that why graph show only one pont because x_test have only one pont.  

In [64]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# ---------- 3D SCATTER (Actual Data) ----------
fig = px.scatter_3d(
    df,
    x='Squarefootage',
    y='Badroom',
    z='Price',
    color='Location_Suburb',  # encoded column from your df
    title='3D Scatter with Prediction Plane'
)

# ---------- GRID FOR PLANE ----------
sq = np.linspace(df['Squarefootage'].min(), df['Squarefootage'].max(), 20)
br = np.linspace(df['Badroom'].min(), df['Badroom'].max(), 20)
SQ, BR = np.meshgrid(sq, br)

# ---------- CONSTANT FEATURES ----------
BA = np.full(SQ.shape, df['Bathroom'].mean())

# Location columns (from your one-hot encoding with drop_first=True)
LOC_CITY = np.zeros(SQ.shape)
LOC_RURAL = np.zeros(SQ.shape)  # Subury=0, City=0, default plane

# ---------- CREATE PLANE INPUT ----------
X_plane = np.c_[
    SQ.ravel(),
    BR.ravel(),
    BA.ravel(),
    LOC_CITY.ravel(),
    LOC_RURAL.ravel()
]

# ---------- PREDICT ----------
y_pred = model.predict(X_plane).reshape(SQ.shape)

# ---------- ADD PREDICTION PLANE ----------
fig.add_trace(
    go.Surface(
        x=SQ,
        y=BR,
        z=y_pred,
        opacity=1,
        showscale=False,
        name='Prediction Plane'
    )
)




c:\NUMPY_YT\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning:

X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names



In [81]:
# user input 
print("enter the details for house prediction:")
sq_footage = float(input("Square Footage:"))
bedrooms = int(input("Number of Bedrooms:"))
bathrooms = float(input("Number of Bathrroms:"))
location = input("Location(Suburb/City/Rural):")

enter the details for house prediction:


In [ ]:
input_location =[0,0]# [Rural, Subury] #avoid dummy variable trap 

if location =='Rural':
   input_location[0]=1   

elif location == 'Suburb':
   input_location[1]=1




user_input = pd.DataFrame({'Squarefootage':[sq_footage],
      'Badroom':[bedrooms],
      'Bathroom':[bathrooms],
      'Location_Rural':[input_location[0]],'Location_Suburb':[input_location[1]]
      })


#make the prediction
predicted_price = model.predict(user_input)
print(f"predicted price for the house :{predicted_price[0]}")        


predicted price for the house :279277.06003189087
